# Advanced Fluorescence Spectroscopy Analysis for Olive Oil Aging

This notebook demonstrates a complete, end-to-end workflow for analyzing fluorescence spectroscopy data from olive oil aging experiments using `RamanSPy`, `pandas`, `plotly`, and `scikit-learn`.

### Workflow Overview:
1.  **Mock Data Generation**: Create a realistic dataset if one doesn't exist.
2.  **Data Loading**: Load multi-dimensional spectral data with robust metadata parsing.
3.  **Preprocessing & Quality Control**: Apply a standard pipeline (denoising, baseline correction, normalization) and filter low-quality spectra.
4.  **Advanced Visualization**: Generate a variety of plots, including 2D/3D interactive spectra, heatmaps, and aging progression dashboards.
5.  **Statistical & Peak Analysis**: Extract key metrics (peak intensity, wavelength, SNR) and track how they evolve over time.
6.  **Machine Learning**: Use PCA and a RandomForestClassifier to classify spectra based on their aging step.
7.  **Data Export**: Save all processed data, statistical results, and high-resolution figures.

## 0. Initial Setup and Imports

In [1]:
# --- Core Libraries ---
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

# --- RamanSPy ---
import ramanspy as rp
from ramanspy import SpectralContainer
from ramanspy.preprocessing import Pipeline
from ramanspy.preprocessing.denoise import SavGol
from ramanspy.preprocessing.baseline import ASLS
from ramanspy.preprocessing.normalise import Vector

# --- Interactive Plotting ---
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

# --- Scientific & ML ---
from scipy import signal
from scipy.stats import norm, pearsonr
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# --- Setup & Configuration ---
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['font.size'] = 12

print("Libraries imported and notebook configured.")

Libraries imported and notebook configured.


## 1. Mock Data Generation (Optional)

This section contains a utility function to generate mock data that mimics the expected structure. This makes the notebook fully runnable without needing the original dataset. If you have your data in the `data/extracted` directory, you can skip running this cell.

In [2]:
def generate_mock_data(base_path="data/extracted", aging_steps=5, spectra_per_file=3, files_per_step=4):
    """Generates mock fluorescence spectroscopy data.
    
    Creates a directory structure and CSV files with simulated spectral data.
    The spectral peak shifts and broadens with each aging step.
    """
    base_path = Path(base_path)
    if base_path.exists():
        print(f"Data directory '{base_path}' already exists. Skipping generation.")
        return
    
    print(f"Generating mock data in '{base_path}'...")
    wavelengths = np.linspace(400, 800, 1024) # Wavelengths in nm

    for i in range(aging_steps):
        step_dir = base_path / f"AS{i}" / "Fluorescence"
        step_dir.mkdir(parents=True, exist_ok=True)
        
        for j in range(files_per_step):
            # Simulate aging effect: peak shifts to longer wavelengths and broadens
            peak_center = 450 + i * 15  # Peak center shifts right
            peak_width = 25 + i * 5     # Peak gets broader
            peak_height = 2000 - i * 150 # Intensity decreases

            df = pd.DataFrame({'Wavelength_nm': wavelengths})
            for k in range(spectra_per_file):
                # Generate a spectrum using a normal distribution (Gaussian peak)
                noise = np.random.normal(0, 50, len(wavelengths))
                baseline = np.linspace(100, 50, len(wavelengths)) + np.random.rand() * 20
                spectrum = peak_height * norm.pdf(wavelengths, peak_center + np.random.randn()*5, peak_width) + baseline + noise
                df[f'Intensity_{k+1}'] = spectrum.astype(int)
            
            # Create a realistic filename
            filename = f"20250621_150{j}_AS{i}_Q1K2V{j}U{k}.csv"
            df.to_csv(step_dir / filename, index=False)
            
    print("Mock data generation complete.")

# Run the data generator
generate_mock_data()

Data directory 'data/extracted' already exists. Skipping generation.


## 2. Data Loading and Preprocessing Utilities

Here, we define classes to handle loading, preprocessing, and quality control of the spectral data.

In [5]:
from hyrcania.preprocessing import baseline_correction


class FluorescenceDataLoader:
    """Advanced data loader for fluorescence spectroscopy data."""
    
    def __init__(self, data_path="data/extracted"):
        self.data_path = Path(data_path)
        if not self.data_path.exists():
            raise FileNotFoundError(f"Data path {self.data_path} not found. Please generate mock data or provide a valid path.")
        self.aging_steps = sorted([d for d in self.data_path.iterdir() if d.is_dir()])
        self.encodings = ['utf-8', 'latin-1', 'cp1252']

    def _load_csv_with_encoding(self, file_path):
        """Load CSV file with automatic encoding detection."""
        for encoding in self.encodings:
            try:
                return pd.read_csv(file_path, encoding=encoding)
            except UnicodeDecodeError:
                continue
        raise ValueError(f"Could not read {file_path} with any of the specified encodings.")

    def _extract_metadata_from_filename(self, filename):
        """Extract metadata from filename pattern (e.g., 20210512_0752_AS0_Q1K2V1U0.csv)."""
        parts = Path(filename).stem.split('_')
        return {'date': parts[0], 'time': parts[1], 'aging_step': parts[2], 'sample_code': parts[3]}

    def load_aging_step_data(self, aging_step_path):
        """Load all fluorescence data for a single aging step into a list of dictionaries."""
        fluorescence_dir = aging_step_path / "Fluorescence"
        files = sorted(fluorescence_dir.glob("*.csv")) if fluorescence_dir.exists() else []
        data_list = []
        
        for file_path in files:
            try:
                df = self._load_csv_with_encoding(file_path)
                metadata = self._extract_metadata_from_filename(file_path.name)
                
                if len(df.columns) < 2:
                    continue

                wavelengths = df.iloc[:, 0].values
                for i, col in enumerate(df.columns[1:]):
                    intensities = df[col].values
                    data_list.append({
                        'wavelengths': wavelengths,
                        'intensities': intensities,
                        'metadata': {**metadata, 'spectrum_index': i, 'source_file': file_path.name}
                    })
            except Exception as e:
                print(f"Error loading {file_path}: {e}")
        return data_list

    def create_spectral_container(self, data_list):
        """Convert loaded data into a RamanSPy SpectralContainer."""
        if not data_list:
            return None
        
        spectral_axis = data_list[0]['wavelengths']
        # Stack all intensity arrays into a 2D numpy array (n_spectra, n_wavelengths)
        intensities = np.vstack([d['intensities'] for d in data_list])
        metadata_list = [d['metadata'] for d in data_list]
        
        return SpectralContainer(intensities, spectral_axis, metadata=metadata_list)

class AdvancedPreprocessor:
    """A complete preprocessing pipeline for fluorescence data."""
    
    def __init__(self):
        # Define a robust preprocessing pipeline
        self.pipeline = Pipeline([
            ('savgol_denoise', SavGol(window_length=11, polyorder=3)),
            ('asls_baseline', ASLS()),
            ('vector_normalise', Vector())
        ])

    def apply_pipeline(self, spectral_container):
        """Apply the full preprocessing pipeline to a container."""
        return self.pipeline.apply(spectral_container)
    
    def quality_control(self, container, snr_threshold=3.0):
        """Filter spectra based on Signal-to-Noise Ratio (SNR)."""
        snr_values = []
        for spectrum in container:
            # Use raw data for SNR calculation to avoid filtering out weak signals pre-correction
            signal_power = np.mean(spectrum.spectral_data)
            noise_power = np.std(spectrum.spectral_data)
            snr = (signal_power / noise_power) if noise_power > 0 else 0
            snr_values.append(snr)
        
        good_indices = [i for i, snr in enumerate(snr_values) if snr > snr_threshold]
        print(f"QC: Kept {len(good_indices)} of {len(container)} spectra (SNR threshold > {snr_threshold})")
        return container[good_indices], np.array(snr_values)

## 3. Load, Preprocess, and Verify Data

Now, we'll instantiate our helper classes and run the main data loading and preprocessing workflow.

In [6]:
try:
    loader = FluorescenceDataLoader()
    preprocessor = AdvancedPreprocessor()

    aging_containers_raw = {}
    aging_containers_processed = {}

    print(f"Found {len(loader.aging_steps)} aging steps. Loading...")
    # Load all aging steps, but you can slice for testing e.g., loader.aging_steps[:3]
    for step_path in loader.aging_steps:
        aging_label = step_path.name
        print(f"\n--- Processing {aging_label} ---")
        
        # Load raw data into list of dicts
        raw_data_list = loader.load_aging_step_data(step_path)
        if not raw_data_list:
            print(f"No data found for {aging_label}")
            continue

        # Create a raw spectral container
        container_raw = loader.create_spectral_container(raw_data_list)
        aging_containers_raw[aging_label] = container_raw
        print(f"Loaded {len(container_raw)} raw spectra.")

        # Perform quality control
        container_clean, snr_values = preprocessor.quality_control(container_raw, snr_threshold=5)
        if len(container_clean) == 0:
            print(f"All spectra for {aging_label} were filtered out by QC.")
            continue

        # Apply preprocessing pipeline
        container_processed = preprocessor.apply_pipeline(container_clean)
        aging_containers_processed[aging_label] = container_processed
        print(f"Successfully processed {len(container_processed)} spectra.")

    # Convert to lists for easier access
    processed_containers = list(aging_containers_processed.values())
    processed_labels = list(aging_containers_processed.keys())

    print(f"\nWorkflow complete. Processed data for {len(processed_containers)} aging steps.")

except FileNotFoundError as e:
    print(e)
    processed_containers = []

Found 10 aging steps. Loading...

--- Processing Aging Step 0 ---


: 

: 

: 

## 4. Advanced Visualization
This class encapsulates all plotting functions. **Correction Note:** Methods that generate Matplotlib plots now return the `figure` object. This allows us to either display the plot directly in the notebook (`plt.show()`) or save it to a file without showing it, fixing the bug in the original `DataExporter`.

In [ ]:
class AdvancedVisualizer:
    """Advanced visualization tools for fluorescence spectroscopy."""
    
    def __init__(self, containers, labels):
        self.containers = containers
        self.labels = labels
        self.colors = plt.cm.viridis(np.linspace(0, 1, len(labels)))
    
    def plot_spectrum_comparison(self, title="Spectrum Comparison"):
        """Compare mean spectra of all aging steps with confidence intervals."""
        fig, ax = plt.subplots(figsize=(14, 8))
        
        for i, container in enumerate(self.containers):
            mean_spec = container.mean.spectral_data
            std_spec = np.std(container.spectral_data, axis=0)
            ax.plot(container.spectral_axis, mean_spec, label=self.labels[i], color=self.colors[i], lw=2.5)
            ax.fill_between(container.spectral_axis, mean_spec - std_spec, mean_spec + std_spec, 
                            color=self.colors[i], alpha=0.2)
        
        ax.set_title(title, fontsize=16, fontweight='bold')
        ax.set_xlabel('Wavelength (nm)', fontsize=12)
        ax.set_ylabel('Normalized Intensity (a.u.)', fontsize=12)
        ax.legend()
        ax.grid(True, which='both', linestyle='--', linewidth=0.5)
        plt.tight_layout()
        return fig

    def plot_heatmap(self, container_index=0, title_prefix="Spectral Heatmap"):
        """Create a heatmap for a specific aging step."""
        container = self.containers[container_index]
        label = self.labels[container_index]
        fig, ax = plt.subplots(figsize=(14, 8))
        
        im = ax.imshow(container.spectral_data, aspect='auto', cmap='viridis',
                       extent=[container.spectral_axis[0], container.spectral_axis[-1], 0, len(container)])
        
        ax.set_title(f"{title_prefix} - {label}", fontsize=16, fontweight='bold')
        ax.set_xlabel('Wavelength (nm)', fontsize=12)
        ax.set_ylabel('Spectrum Index', fontsize=12)
        
        cbar = fig.colorbar(im, ax=ax)
        cbar.set_label('Normalized Intensity')
        plt.tight_layout()
        return fig
        
    def plot_interactive_3d_surface(self, container_index=0, title_prefix="Interactive 3D Surface"):
        """Create an interactive 3D surface plot using Plotly."""
        container = self.containers[container_index]
        label = self.labels[container_index]
        
        fig = go.Figure(data=[go.Surface(
            z=container.spectral_data,
            x=container.spectral_axis,
            y=np.arange(len(container)),
            colorscale='Viridis', cmin=container.spectral_data.min(), cmax=container.spectral_data.max(),
            colorbar=dict(title='Intensity')
        )])
        
        fig.update_layout(
            title={'text': f"{title_prefix} - {label}", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
            scene=dict(
                xaxis_title='Wavelength (nm)',
                yaxis_title='Spectrum Index',
                zaxis_title='Normalized Intensity'
            ),
            width=800, height=700
        )
        fig.show()
        
    def plot_pca_analysis(self):
        """Perform and visualize PCA on all combined data."""
        all_data = np.vstack([c.spectral_data for c in self.containers])
        all_labels = []
        for i, c in enumerate(self.containers):
            all_labels.extend([self.labels[i]] * len(c))
        
        if all_data.shape[0] < 3:
            print("PCA requires more data points.")
            return
        
        # Scale data before PCA
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(all_data)
        
        pca = PCA(n_components=3)
        X_pca = pca.fit_transform(X_scaled)
        
        print(f"Total explained variance by 3 PCs: {np.sum(pca.explained_variance_ratio_):.2%}")

        df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2', 'PC3'])
        df_pca['Aging Step'] = all_labels

        fig = px.scatter_3d(
            df_pca, x='PC1', y='PC2', z='PC3', 
            color='Aging Step', 
            title='PCA of Olive Oil Spectra Across Aging Steps',
            labels={'PC1': f'PC1 ({pca.explained_variance_ratio_[0]:.1%})',
                    'PC2': f'PC2 ({pca.explained_variance_ratio_[1]:.1%})',
                    'PC3': f'PC3 ({pca.explained_variance_ratio_[2]:.1%})'}
        )
        fig.update_traces(marker=dict(size=5))
        fig.show()

### 4.1. Generating Visualizations

Let's create the plots. Note that interactive plots (`plotly`) will be displayed directly, while static plots (`matplotlib`) will be shown via `plt.show()`.

In [ ]:
if processed_containers:
    visualizer = AdvancedVisualizer(processed_containers, processed_labels)

    # --- Static Plots (Matplotlib) ---
    print("Generating static plots...")
    
    # Plot 1: Mean Spectra Comparison
    fig1 = visualizer.plot_spectrum_comparison(title="Mean Fluorescence Spectra Across Aging Steps")
    plt.show()

    # Plot 2: Heatmap of the first aging step
    fig2 = visualizer.plot_heatmap(container_index=0)
    plt.show()
    
    # --- Interactive Plots (Plotly) ---
    print("\nGenerating interactive plots...")
    
    # Plot 3: Interactive 3D Surface
    visualizer.plot_interactive_3d_surface(container_index=0)
    
    # Plot 4: Interactive PCA plot
    visualizer.plot_pca_analysis()
else:
    print("Skipping visualization as no data was processed.")

## 5. Statistical and Peak Analysis

These classes provide methods to extract quantitative metrics from the spectra and analyze how they change over time.

In [ ]:
class StatisticalAnalyzer:
    """Tools for extracting statistical metrics from spectral data."""
    
    def _calculate_snr(self, data):
        mean_spectrum = np.mean(data, axis=0)
        signal = np.mean(mean_spectrum)
        noise = np.std(mean_spectrum)
        return signal / noise if noise > 0 else 0

    def calculate_metrics(self, container):
        """Calculate a dictionary of key metrics for a container."""
        mean_spec = container.mean.spectral_data
        
        metrics = {
            'mean_snr': self._calculate_snr(container.spectral_data),
            'peak_intensity': np.max(mean_spec),
            'peak_wavelength': container.spectral_axis[np.argmax(mean_spec)],
            'total_intensity': np.mean(np.sum(container.spectral_data, axis=1))
        }
        return metrics

    def analyze_all_steps(self, containers, labels):
        """Analyze all aging steps and return a DataFrame of metrics."""
        all_metrics = []
        for i, container in enumerate(containers):
            metrics = self.calculate_metrics(container)
            metrics['aging_step'] = labels[i]
            all_metrics.append(metrics)
        
        return pd.DataFrame(all_metrics).set_index('aging_step')

class PeakAnalyzer:
    """Advanced peak analysis for fluorescence spectra."""

    def find_peaks(self, spectrum, wavelengths, height=0.1, distance=10):
        """Find peaks in a single spectrum using scipy.signal.find_peaks."""
        peaks, properties = signal.find_peaks(spectrum, height=height, distance=distance)
        return pd.DataFrame({
            'wavelength': wavelengths[peaks],
            'intensity': spectrum[peaks],
            'prominence': properties.get('prominences', [None]*len(peaks))
        })

    def analyze_peak_evolution(self, containers, labels):
        """Analyze how the main peak evolves across all aging steps."""
        peak_info = []
        for i, container in enumerate(containers):
            mean_spec = container.mean.spectral_data
            # Find the single most prominent peak in the mean spectrum
            peaks_df = self.find_peaks(mean_spec, container.spectral_axis)
            if not peaks_df.empty:
                main_peak = peaks_df.loc[peaks_df['intensity'].idxmax()]
                info = main_peak.to_dict()
                info['aging_step'] = labels[i]
                peak_info.append(info)
        
        return pd.DataFrame(peak_info).set_index('aging_step')

### 5.1. Running Analysis and Visualizing Results

In [ ]:
if processed_containers:
    # Perform statistical and peak analysis
    stat_analyzer = StatisticalAnalyzer()
    stat_results_df = stat_analyzer.analyze_all_steps(processed_containers, processed_labels)
    
    peak_analyzer = PeakAnalyzer()
    peak_results_df = peak_analyzer.analyze_peak_evolution(processed_containers, processed_labels)
    
    # Combine results for display
    results_df = pd.concat([stat_results_df, peak_results_df.drop(columns=['intensity'])], axis=1)
    results_df.rename(columns={'wavelength': 'main_peak_wavelength'}, inplace=True)

    print("--- Combined Statistical and Peak Analysis Results ---")
    display(results_df)

    # Visualize the results
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Evolution of Spectral Metrics Over Aging Steps', fontsize=18, fontweight='bold')
    axes = axes.ravel()

    results_df['peak_intensity'].plot(ax=axes[0], style='o-', title='Peak Intensity', colormap='viridis', lw=2)
    results_df['main_peak_wavelength'].plot(ax=axes[1], style='o-', title='Peak Wavelength', colormap='plasma', lw=2)
    results_df['total_intensity'].plot(ax=axes[2], style='s-', title='Average Total Intensity', colormap='magma', lw=2)
    results_df['prominence'].plot(ax=axes[3], style='^-', title='Main Peak Prominence', colormap='cividis', lw=2)

    for ax in axes:
        ax.set_ylabel('')
        ax.set_xlabel('Aging Step')
        ax.grid(True, linestyle='--')

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()
else:
    print("Skipping statistical analysis as no data was processed.")


## 6. Machine Learning Integration

Here we attempt to build a classifier that can distinguish between different aging steps based purely on the spectral data. This demonstrates the potential for automated quality assessment.

In [ ]:
class MLIntegrator:
    """Machine learning integration for spectral classification."""

    def __init__(self, containers, labels):
        self.containers = containers
        self.labels = labels
        self.scaler = StandardScaler()
        # Use PCA to reduce dimensionality while keeping 95% of variance
        self.pca = PCA(n_components=0.95)
        self.model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

    def prepare_data(self):
        """Prepare feature matrix (X) and target vector (y)."""
        X = np.vstack([c.spectral_data for c in self.containers])
        y = []
        for i, c in enumerate(self.containers):
            y.extend([self.labels[i]] * len(c))
        return X, np.array(y)

    def run_classification_pipeline(self):
        """Run the full train-test-evaluate pipeline."""
        X, y = self.prepare_data()
        
        # Split data, stratifying to handle imbalanced classes
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=42, stratify=y
        )
        
        # Fit scaler on training data only
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)

        # Fit PCA on training data only
        X_train_pca = self.pca.fit_transform(X_train_scaled)
        X_test_pca = self.pca.transform(X_test_scaled)

        print(f"Original feature count: {X.shape[1]}")
        print(f"PCA feature count: {self.pca.n_components_} (retaining {self.pca.explained_variance_ratio_.sum():.2%} variance)")
        
        # Train model
        self.model.fit(X_train_pca, y_train)
        
        # Evaluate
        y_pred = self.model.predict(X_test_pca)
        
        print("\n--- Classification Report ---")
        report = classification_report(y_test, y_pred, target_names=np.unique(y))
        print(report)
        return report

In [ ]:
if processed_containers and len(processed_containers) > 1:
    ml_integrator = MLIntegrator(processed_containers, processed_labels)
    classification_report_str = ml_integrator.run_classification_pipeline()
else:
    print("Skipping ML integration: requires at least 2 processed aging steps.")

## 7. Export and Save Results
This class handles saving the processed data, statistical analysis, and visualizations to an `output` directory.

In [ ]:
class DataExporter:
    """Export processed data, stats, and visualizations."""
    
    def __init__(self, output_dir="output"):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)
        print(f"Results will be saved to '{self.output_dir}'")

    def save_processed_data(self, containers, labels):
        """Save processed spectral data to separate CSV files."""
        data_dir = self.output_dir / "processed_data"
        data_dir.mkdir(exist_ok=True)
        for container, label in zip(containers, labels):
            df = pd.DataFrame(container.spectral_data, columns=container.spectral_axis.astype(str))
            filepath = data_dir / f"processed_spectra_{label}.csv"
            df.to_csv(filepath, index_label='Spectrum_Index')
        print(f"Saved processed spectral data to '{data_dir}'.")

    def save_statistics(self, stats_df):
        """Save the statistical summary DataFrame to a CSV."""
        filepath = self.output_dir / "statistical_summary.csv"
        stats_df.to_csv(filepath)
        print(f"Saved statistical summary to '{filepath}'.")

    def save_visualizations(self, visualizer_instance):
        """Save key Matplotlib visualizations as high-resolution images."""
        vis_dir = self.output_dir / "visualizations"
        vis_dir.mkdir(exist_ok=True)
        
        # Save Spectrum Comparison Plot
        fig_comp = visualizer_instance.plot_spectrum_comparison()
        fig_comp.savefig(vis_dir / "spectrum_comparison.png", dpi=300, bbox_inches='tight')
        plt.close(fig_comp) # Close figure to free memory

        # Save Heatmap of first step
        fig_heat = visualizer_instance.plot_heatmap(container_index=0)
        fig_heat.savefig(vis_dir / "heatmap_first_step.png", dpi=300, bbox_inches='tight')
        plt.close(fig_heat)
        
        print(f"Saved visualizations to '{vis_dir}'.")

In [ ]:
if processed_containers:
    exporter = DataExporter()
    
    # Export processed spectra
    exporter.save_processed_data(processed_containers, processed_labels)
    
    # Export statistical results
    if 'results_df' in locals():
        exporter.save_statistics(results_df)
    
    # Export visualizations
    if 'visualizer' in locals():
        exporter.save_visualizations(visualizer)
        
    print("\n--- Export complete! ---")
else:
    print("Skipping export as no data was processed.")

## 8. Summary and Conclusions

This notebook successfully executed a full analysis pipeline. Key findings often show a shift in peak wavelength and a change in spectral shape as the olive oil ages, which is quantifiable through statistical analysis and classifiable by machine learning models. The generated outputs in the `output` directory provide a comprehensive record of this analysis.